# 0. 환경설정 및 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_path = '/Users/jeongwonyoo/Desktop/portfolio/projects/ImprovingJobPlatformDropoffRates/log_2023.csv'
log_2023_df = pd.read_csv(data_path, index_col=0)

# 1. Cleaning Data

In [4]:
log_2023_df.shape

(7187783, 6)

In [21]:
log_2023_df.head(3)

,user_uuid,URL,timestamp,date,response_code,method
0,5ce8f5ca-3476-4623-a60c-00c98eef3b62,@user_id,2023-12-29 13:19:50.230356 UTC,2023-12-29,200,GET
1,5ce8f5ca-3476-4623-a60c-00c98eef3b62,api/users/notifications/mark_read?id=6425064&_...,2023-12-29 13:20:17.848762 UTC,2023-12-29,200,GET
2,5ce8f5ca-3476-4623-a60c-00c98eef3b62,jobs/id/id_title,2023-12-29 13:22:22.277796 UTC,2023-12-29,200,GET


## Change Data Type : date column -> datetime

In [22]:
log_2023_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7187783 entries, 0 to 7187782
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_uuid      object
 1   URL            object
 2   timestamp      object
 3   date           object
 4   response_code  int64 
 5   method         object
dtypes: int64(1), object(5)
memory usage: 383.9+ MB


In [23]:
log_2023_df['date'] = pd.to_datetime(log_2023_df['date'])

In [24]:
log_2023_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7187783 entries, 0 to 7187782
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_uuid      object        
 1   URL            object        
 2   timestamp      object        
 3   date           datetime64[ns]
 4   response_code  int64         
 5   method         object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 383.9+ MB


## Cleaning response code

In [25]:
log_2023_df['response_code'].value_counts()

response_code
200    7033181
302     102602
400      29427
404      15574
409       4834
500       1056
405        479
401        475
301         57
403         57
503         41
Name: count, dtype: int64

2xx : Success

3xx : Redirection, A redirection must occur for the request to be considered complete (e.g., redirection to an external web server)

4xx : Client Error

5xx : Server Error

→ To remove errors from user activity logs, 4xx and 5xx status codes are excluded

In [26]:
delete_response_code = log_2023_df['response_code'].isin([404, 400, 409, 500, 401, 405, 503, 403])
log_2023_df = log_2023_df[~delete_response_code]

In [27]:
log_2023_df['response_code'].value_counts()

response_code
200    7033181
302     102602
301         57
Name: count, dtype: int64

## Cleaning URL

### Remove nan value from URL column : (369,434 rows)
- Logs without URL information are considered data errors

In [28]:
log_2023_df.dropna(inplace=True)
log_2023_df.isna().sum()

user_uuid        0
URL              0
timestamp        0
date             0
response_code    0
method           0
dtype: int64

**Primary URL keyword-based classification**
- **Job seekers** : /search/, /users/, @user_id, jobs/id/, setting, timeline, recommend, guided action, apply_progress, suggest,
verify, continue?next=/@, help/id, verify_phone, api/job_offer, api/post/id/, remove, api/project/form_data/media,
api/jobs/user_filter/id, user_received, continue?next=/jobs, api/jobs/widget/widget_templates, jobs?specialty=, people?rel=1,
people?keywords=, api/references, api/jobs/collections/template, api/ask-manager/id, jobs?location, api/media/id/form,
api/specialties/id/follow_button, jobs?career_type, api/people/template, /signup/

- **Employers** : jobs?page=&job, pricing, api/jobs/form_data/media, api/page/id/form

- **Others** : people, zip_code?index=, api/application_thread_comment/id, app,
password_reset, email_verify\?code=, people\?school, people\?job=1&location=, api/page/id/view, api/page/id/form

### Romove admin's logs

<mark> Since our analysis target is users, logs containing 'admin' are excluded from the removal logs
- "api/search/people/job_title?name=멸치TV A(melchi App)" and "admin관리 시스템 메인 기획 및 운영 매니저&_=1699526824841",
  
  These logs may correspond to administrators or employer representatives; however, since they are not users, they are excluded from the analysis

In [29]:
condition_admin = log_2023_df['URL'].str.contains('admin')
log_2023_df.loc[condition_admin, 'URL'].value_counts()

URL
api/search/specialty?name=admin                                                                                                                                  3
api/search/people/job_title?name=멸치TV App(melchi TV) 및 admin 시스템 메인 기획 및 운영 매니저&_=1699526824845                                                                  1
api/search/product?name=Designing an extension for Learning Management Software (LMS) to introduce a more efficient group task administration&_=1682819256962    1
api/search/product?name=LUXON admin&_=1684205620009                                                                                                              1
api/search/product?name=LUXON admin-a&_=1684205620011                                                                                                            1
api/search/product?name=LUXON admin &_=1684205850271                                                                                                             1
api/search/product

In [30]:
log_2023_df = log_2023_df[~condition_admin]

### company 제거

<mark> Since our anal`ysis target is users, logs that are clearly associated with companies are excluded from the analysis

```python
- pricing : Payment-related selections are primarily made by companies
- api/jobs/form_data/media : Editing media for job postings
- api/companies/id/ad_stat/progress : Advertisement-related activity
- api/page/id/form : Page form POST method, associated with companies
- api/page/id/view : Page view POST method, associated with companies
- api/people/template : Viewing a specific person’s template
- api/users/id/request_button : Request button, mainly used by companies to request access to information about a person
- jobs/id/applications : Fetching the list of applicants for a job posting
- api/companies/id/form	: Loading the company information input/edit form
```

In [31]:
pattern_company = (
    'pricing|api/jobs/form_data/media|api/companies/id/ad_stat/progress|api/page/id/view|api/people/template|'
    'api/page/id/form|api/users/id/request_button|jobs/id/applications.*|api/companies/id/form'
)
condition_companies = log_2023_df['URL'].str.contains(pattern_company, regex=True, na=False)

In [32]:
log_2023_filtering = log_2023_df[~condition_companies]

In [33]:
log_2023_filtering

,user_uuid,URL,timestamp,date,response_code,method
0,5ce8f5ca-3476-4623-a60c-00c98eef3b62,@user_id,2023-12-29 13:19:50.230356 UTC,2023-12-29,200,GET
1,5ce8f5ca-3476-4623-a60c-00c98eef3b62,api/users/notifications/mark_read?id=6425064&_...,2023-12-29 13:20:17.848762 UTC,2023-12-29,200,GET
2,5ce8f5ca-3476-4623-a60c-00c98eef3b62,jobs/id/id_title,2023-12-29 13:22:22.277796 UTC,2023-12-29,200,GET
3,5ce8f5ca-3476-4623-a60c-00c98eef3b62,suggest?q=epdlxj,2023-12-29 13:21:22.99993 UTC,2023-12-29,200,GET
4,5ce8f5ca-3476-4623-a60c-00c98eef3b62,api/current_guided_action/id,2023-12-29 13:20:19.834724 UTC,2023-12-29,200,POST
...,...,...,...,...,...,...
7187778,167cdb14-d145-4679-b878-66a9a2d32ee9,@user_id,2023-11-14 12:12:26.780872 UTC,2023-11-14,200,GET
7187779,167cdb14-d145-4679-b878-66a9a2d32ee9,@user_id,2023-11-14 12:11:38.859562 UTC,2023-11-14,200,GET
7187780,f5649d90-3016-4747-9924-a05d74ee895e,api/users/id/template,2023-11-14 13:16:19.471324 UTC,2023-11-14,200,POST
7187781,f5649d90-3016-4747-9924-a05d74ee895e,verify_phone?next_url=/%40kkwangpe,2023-11-14 13:16:11.530244 UTC,2023-11-14,200,GET


In [34]:
# Check whether any patterns still remain in log_2023_filtering after removal
has_company_pattern = log_2023_filtering['URL'].str.contains(pattern_company).any()

if has_company_pattern:
    print("⚠️ URLs matching the pattern_company condition still remain in log_2023_filtering.")
    remaining_logs = log_2023_filtering[log_2023_filtering['URL'].str.contains(pattern_company)]
    display(remaining_logs['URL'].unique())
else:
    print("✅ All URLs matching pattern_company have been completely removed from log_2023_filtering.")

✅ All URLs matching pattern_company have been completely removed from log_2023_filtering.


### Remove 'app' URL

<mark> Since our ultimate business goal is application completion on desktop, app traffic is excluded from the analysis.

In [35]:
condition_app = log_2023_df['URL'].isin(['app'])
log_2023_df.loc[condition_app, 'URL'].value_counts()

URL
app    1876
Name: count, dtype: int64

In [36]:
log_2023_filtering = log_2023_df[~condition_app]

In [37]:
log_2023_filtering

,user_uuid,URL,timestamp,date,response_code,method
0,5ce8f5ca-3476-4623-a60c-00c98eef3b62,@user_id,2023-12-29 13:19:50.230356 UTC,2023-12-29,200,GET
1,5ce8f5ca-3476-4623-a60c-00c98eef3b62,api/users/notifications/mark_read?id=6425064&_...,2023-12-29 13:20:17.848762 UTC,2023-12-29,200,GET
2,5ce8f5ca-3476-4623-a60c-00c98eef3b62,jobs/id/id_title,2023-12-29 13:22:22.277796 UTC,2023-12-29,200,GET
3,5ce8f5ca-3476-4623-a60c-00c98eef3b62,suggest?q=epdlxj,2023-12-29 13:21:22.99993 UTC,2023-12-29,200,GET
4,5ce8f5ca-3476-4623-a60c-00c98eef3b62,api/current_guided_action/id,2023-12-29 13:20:19.834724 UTC,2023-12-29,200,POST
...,...,...,...,...,...,...
7187778,167cdb14-d145-4679-b878-66a9a2d32ee9,@user_id,2023-11-14 12:12:26.780872 UTC,2023-11-14,200,GET
7187779,167cdb14-d145-4679-b878-66a9a2d32ee9,@user_id,2023-11-14 12:11:38.859562 UTC,2023-11-14,200,GET
7187780,f5649d90-3016-4747-9924-a05d74ee895e,api/users/id/template,2023-11-14 13:16:19.471324 UTC,2023-11-14,200,POST
7187781,f5649d90-3016-4747-9924-a05d74ee895e,verify_phone?next_url=/%40kkwangpe,2023-11-14 13:16:11.530244 UTC,2023-11-14,200,GET


In [38]:
# Check whether 'app' still remains after removal
has_app = log_2023_filtering['URL'].isin(['app']).any()

if has_app:
    print("⚠️ The 'app' URL still remains in log_2023_filtering.")
    print(log_2023_filtering[log_2023_filtering['URL'] == 'app'])
else:
    print("✅ The 'app' URL has been completely removed from log_2023_filtering.")

✅ The 'app' URL has been completely removed from log_2023_filtering.


## Analysis Period: April 2023 – September 2023
- Months with unusually high (early-year) or low (year-end) volumes of job postings may distort trend analysis; therefore, a six-month period excluding these extremes was selected for analysis

In [39]:
log_2023_filtering.shape

(6860968, 6)

In [40]:
log_2023_filtering = log_2023_filtering[
    (log_2023_filtering['date'].dt.month >= 4) &
    (log_2023_filtering['date'].dt.month <= 9)
]

In [41]:
log_2023_filtering.shape

(3528948, 6)

# 2. Funnel Stage Classification

In [5]:
data_path = '/Users/jeongwonyoo/Desktop/portfolio/projects/ImprovingJobPlatformDropoffRates/log_2023_filtering.csv'
log = pd.read_csv(data_path)

## Stage 1: Entry


[URLs classified as Stage 1]

- setting?utm_source=notification&utm_medium=ema... : Accessed via an email push notification
  
- api/job_offer/ : Clicked to view an offer received
  
- ?user_received : Marketing tracking info indicating that a specific user ID received this link
  
- signup : Sign-up (account registration)
  
- api/recommend_specialty : Shows personalized job postings based on recommended specialties (log for preparing exploration content)
  
- api/current_guided_action/id : Returns the next recommended action for the user ("Try doing this next!")

In [6]:
pattern_step1 = 'setting\?utm_source|api/job_offer|user_received|signup|api/recommend_specialty|widget|api/current_guided_action/id'
condition_step1 = log['URL'].str.contains(pattern_step1)
condition_step1.sum()

np.int64(281401)

In [7]:
view_step1 = log.loc[condition_step1, 'URL']

In [8]:
view_step1.value_counts()

URL
api/recommend_specialty                                 177238
api/current_guided_action/id                             80168
api/jobs/widget/widget_templates                          7155
signup/detail                                             1775
signup/step2/career                                       1527
                                                         ...  
api/job_offer/id/modal?type=received&_=1686188677172         1
api/job_offer/id/modal?type=received&_=1685366676696         1
api/job_offer/id/modal?type=received&_=1685366786065         1
api/job_offer/id/modal?type=received&_=1685407683749         1
api/job_offer/id/modal?type=received&_=1684126997390         1
Name: count, Length: 3970, dtype: int64

## Stage 2 : Resume Creation

- Phone and email verification is included in this stage



[URLs classified as Stage 2]

- setting : Job seeker directly accesses their settings page
  
- email_verify?code : Email verification
  
- api/users/id/specialty, api/users/id/form : Logs where the user edits their personal information
  
- @user_id/resume/step : Resume writing steps (the user’s own resume, independent of any job posting)
  
- api/users/id/career/id : Viewing or editing the user’s own career history
  
- api/users/id/project : User builds or edits their portfolio
  
- api/users/id/resume/step : Writing the user’s resume
  
- @user_id/resume : User views their own resume
  
- @user_id : Viewing the user’s profile (generally, @user_id is designed to refer to the user’s own account)
  
- api/users/id/ : Activities related to writing, editing, or viewing the user’s resume
  
- api/guided_action/add : Prompting the user to add information to their profile
  
- api/projects/id/media/add : Adding media files to a project
  
- api/users/id/template : Job seeker loads a self-introduction template
  
- api/media/id/form : Job seeker uploads their own image files
  
- api/project/form_data/media : Uploading media files for the user’s own project
  
- api/verify/education/id : When using the POST method, the job seeker submits and verifies their own information; classified as the resume-writing stage (mid-funnel) in funnel analysis
  
- verify_phone? : Page for phone number verification


In [9]:
# Conditions that should fall under Stage 5 but overlap
# (editing one’s own experience during the application-writing stage)
condition_except_step5 = log['URL'].isin(['api/users/id/experience/form?type=apply'])
condition_except_step5.sum()

np.int64(77161)

In [10]:
pattern_step2 = (
    'email_verify\?code|api/users/id/specialty|api/users/id/form|@user_id/resume/step|api/users/id/career/id|api/users/id/project|'
    'api/users/id/resume/step|@user_id/resume|@user_id|api/users/id|api/guided_action/add|api/projects/id/media/add|api/users/id/template|'
    'api/media/id/form|api/project/form_data/media|api/verify/education/id|verify_phone'
)
condition_step2 = (
    ~condition_step1 & ~condition_except_step5 &
    (log['URL'].str.contains(pattern_step2) | log['URL'].isin(['setting']))
)
condition_step2.sum()

np.int64(958338)

In [11]:
view_step2 = log.loc[condition_step2, 'URL']
view_step2.value_counts()

URL
api/users/id/template                                        276723
@user_id                                                     213019
@user_id/applications                                         69553
api/users/id/specialty                                        27407
@user_id/resume                                               25346
                                                              ...  
api/users/id/reference/template?sent=true&_=1687249132333         1
api/users/id/reference/template?sent=true&_=1692003218315         1
api/users/id/reference/template?_=1692003218316                   1
api/users/id/career/id/form?_=1690768894672                       1
api/users/id/profile_image/facebook?_=1681892262206               1
Name: count, Length: 101897, dtype: int64

## Stage 3 : Exploration

[URLs classified as Stage 3]

- timeline : A list of personalized information shown on the job seeker’s home screen (recommended job postings, recently viewed ads, bookmarked ads, etc.)
  
- suggest?q=생물 : Autocomplete suggestion for the keyword “생물(bio)” (job seeker)
  
- api/jobs/job_title?page=&q= : Request for job title lists (e.g., job seeker searches for roles of interest)
  
- api/jobs/job_title?keywords= : Request for job posting keywords (job seeker search)
  
- jobs?specialty=SQL : Log where the job seeker filters by the skill “SQL”
  
- api/jobs/collections/template : Log for showing personalized job postings to the job seeker
  
- jobs?location= : Log where the job seeker filters by location
  
- api/specialties/id/follow_button : Job seeker follows or unfollows a specific specialty
  
- api/jobs/user_filter/id : Saving or retrieving the user’s interest filters
  
- search/companies : Searching for companies
  
- jobs?page=&job=1&salary=60000000-&location=서울특... : Job seeker continues exploring while applying multiple filters
  
- jobs, job, companies only : Searching or clicking by job or company filters (not a specific job posting)
  
- help/id : Help page accessed by the job seeker
  
- api/post/id/ : Viewing a community post
  
- people?school : Checking a specific person’s school
  
- people?job=1&location : Exploring a specific person’s job and location
  
- api/companies/id/view : Browsing a company page
  
- companies/company_id/jobs : Exploring job postings from a specific company
  
- companies/company_id : Viewing a specific company’s profile
  
- api/jobs/widget/widget_templates : Log for loading templates that determine how job postings are displayed in the UI
  
- api/companies/id/member_list?oneline=1&offset=3 : Viewing the member list of a specific company
  
- @user_id/job_offer/received : Checking the list of job offers received by the user (self-view)
  
- jobs/job_title : Displaying a list of job postings for a specific role
  
- @user_id/bookmark : User bookmarks (since this is not job/id/bookmark, interpreted as bookmarking companies or similar entities)
  
- api/search/specialty?name=re : Searching for a specific specialty
  
- api/companies/id/bookmark : Bookmarking a specific company
  
- jobs?job=1 : Filtering by specific conditions within job postings
  
- api/comapnies/id/follow_button : Following a specific company
  
- api/search/product : Searching based on a specific product or service

In [12]:
pattern_step3 = (
    'timeline|suggest\?q|api/jobs/job_title|jobs\?specialty|api/jobs/collections/template|'
    'jobs\?location|api/specialties/id/follow_button|user_filter|search/companies|jobs\?page=&job|help/id|api/post/id|'
    'people\?school|people\?job=1&location|api/companies/id/view|companies/company_id/jobs|companies/company_id|widget|'
    'api/companies/id/member_list|@user_id/job_offer/received|jobs/job_title|@user_id/bookmark|'
    'api/search/specialty\?name=re|api/companies/id/bookmark|jobs\?job=|api/comapnies/id/follow_button|api/search/product'
)
condition_step3 = (
    ~condition_step1 & ~condition_step2 &
    (log['URL'].str.contains(pattern_step3) | log['URL'].isin(['jobs', 'people', 'companies', 'job']))
)
condition_step3.sum()

np.int64(977269)

In [13]:
view_step3 = log.loc[condition_step3, 'URL']
view_step3.value_counts()

URL
companies/company_id/jobs                           94031
api/companies/id/view                               88233
companies/company_id                                81063
jobs                                                70972
api/jobs/job_title?job=1                            40001
                                                    ...  
api/search/product?name=NurseEdu&_=1686541680573        1
api/search/companies?name=외주프로젝&_=1686541680577         1
api/search/companies?name=d&_=1686541680574             1
api/jobs/job_title?job=1&keywords=핏펫                    1
api/search/companies?name=아&_=1681887866365             1
Name: count, Length: 197311, dtype: int64

## Stage 4 : Job Posting Viewing (Clicking a Specific Job Posting)

[URLs classified as Stage 4]

- jobs/id/id_title : Clicking a job posting

- people?company : Clicking to view the company’s “people” (employees)

- people?rel=1 : Viewing employee information of a specific company (may be company-related, but likelihood is low)

- people?keywords=크리낙&q= : Log for searching employees by the name “크리낙” (likely a company name)

- api/ask-manager/id : Job seeker asks a question to a manager (company representative)

- api/jobs/id/other_jobs?offset=0&limit=5 : Request to load other job postings related to the currently viewed posting

- api/jobs/id/other_jobs : Loading related job postings

- jobs/id/bookmark : Bookmarking a specific job posting

- api/jobs/id/follow_button : Following the job posting


In [14]:
pattern_step4 = (
    'jobs/id/id_title|people\?company|people\?rel=1|api/ask-manager/id|api/jobs/id/other_jobs\?offset=0&limit=5|api/jobs/id/other_jobs|'
    'jobs/id/bookmark|api/jobs/id/follow_button'
)
condition_step4 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & log['URL'].str.contains(pattern_step4)
)
condition_step4.sum()

np.int64(505955)

In [15]:
view_step4 = log.loc[condition_step4, 'URL']
view_step4.value_counts()

URL
jobs/id/id_title                                                                                244954
api/jobs/id/other_jobs?offset=0&limit=5                                                         182916
api/jobs/id/other_jobs                                                                           36141
api/jobs/id/bookmark                                                                             20041
jobs/id/id_title?utm_source=notification&utm_medium=email&utm_campaign=&utm_content=view_job      3753
                                                                                                 ...  
people?company=39063                                                                                 1
people?company=26691                                                                                 1
people?company=185868                                                                                1
people?company=134195                                                

## Stage 5 : Application Writing

[URLs classified as Stage 5]

- continue?next=/@didwndckd/applications : Login redirection
  
- continue?next=/jobs/132489/apply/step1&token=1... : Log indicating navigation to the job application start page after login
  
- api/references : Logs related to references during the application process
  
- jobs/id/apply/step : Application form writing stage
  
- api/jobs/id/template_oneclick : A type of “one-click apply” button
  
- api/users/id/experience/form?type=apply : Loading the user’s experience information form (within a job posting)


In [17]:
# Conditions that should fall under Stage 5 but overlap
condition_except_step5 = log['URL'].isin(['api/users/id/experience/form?type=apply'])
condition_except_step5.sum()

np.int64(77161)

In [18]:
pattern_step5 = (
    'continue\?next=/@|continue\?next=/jobs/|api/references|jobs/id/apply/step|api/jobs/id/template_oneclick'
)
condition_step5 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & ~condition_step4 & (log['URL'].str.contains(pattern_step5) | condition_except_step5)
)
condition_step5.sum()

np.int64(379921)

In [19]:
view_step5 = log.loc[condition_step5, 'URL']
view_step5.value_counts()

URL
api/users/id/experience/form?type=apply                77161
jobs/id/apply/step1                                    47392
jobs/id/apply/step2                                    41653
api/jobs/id/apply/step2                                37231
api/jobs/id/apply/step1                                36785
                                                       ...  
continue?next=/jobs/144770/apply/step1                     1
continue?next=/@46a24d3f&token=KGx4c2DV4KUukb4uEUJj        1
continue?next=/@kmsmlm2&token=iWUmeF1mE2w60NRD7eZZ         1
continue?next=/@03sadie&token=Hau8CMS3dzefpEY7wK0l         1
continue?next=/@luvluvmarie/resume                         1
Name: count, Length: 6418, dtype: int64

## Stage 6 : Completed

[URLs classified as Stage 6]

- @user_id/applications : Job seeker views submitted applications
  
- apply_progress : Checking the application status or hiring stage of applied jobs
  
- api/remove_application/id : Deleting a job application
  
- jobs/id/apply/complete : Application completed

In [20]:
pattern_step6 = '@user_id/applications|apply_progress|api/remove_application/id|jobs/id/apply/complete'
condition_step6 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & ~condition_step4 & ~condition_step5 & log['URL'].str.contains(pattern_step6)
)
condition_step6.sum()

np.int64(49576)

In [21]:
view_step6 = log.loc[condition_step6, 'URL']
view_step6.value_counts()

URL
jobs/id/apply/complete                14010
api/remove_application/id              3977
api/apply_progress?_=1682273405903        2
api/apply_progress?_=1693305678804        1
api/apply_progress?_=1683971054026        1
                                      ...  
api/apply_progress?_=1687330920280        1
api/apply_progress?_=1687331045217        1
api/apply_progress?_=1687330372240        1
api/apply_progress?_=1687330372231        1
api/apply_progress?_=1680668354414        1
Name: count, Length: 31590, dtype: int64

# 3. Funnel Data Draft Creation

## Creating Funnel Data

In [22]:
# Stage 1
pattern_step1 = 'setting\?utm_source|api/job_offer|user_received|signup|api/recommend_specialty|widget|api/current_guided_action/id'
condition_step1 = log['URL'].str.contains(pattern_step1)
step1 = log[condition_step1]
step1_users = set(step1['user_uuid'])

# Stage 2
condition_except_step5 = log['URL'].isin(['api/users/id/experience/form?type=apply'])
pattern_step2 = (
    'email_verify\?code|api/users/id/specialty|api/users/id/form|@user_id/resume/step|api/users/id/career/id|api/users/id/project|'
    'api/users/id/resume/step|@user_id/resume|@user_id|api/users/id|api/guided_action/add|api/projects/id/media/add|api/users/id/template|'
    'api/media/id/form|api/project/form_data/media|api/verify/education/id|verify_phone'
)
condition_step2 = (
    ~condition_step1 & ~condition_except_step5 &
    (log['URL'].str.contains(pattern_step2) | log['URL'].isin(['setting']))
)
step2 = log[condition_step2 & log['user_uuid'].isin(step1_users)]
step2_users = set(step2['user_uuid'])

# Stage 3
pattern_step3 = (
    'timeline|suggest\?q|api/jobs/job_title|jobs\?specialty|api/jobs/collections/template|'
    'jobs\?location|api/specialties/id/follow_button|user_filter|search/companies|jobs\?page=&job|help/id|api/post/id|'
    'people\?school|people\?job=1&location|api/companies/id/view|companies/company_id/jobs|companies/company_id|widget|'
    'api/companies/id/member_list|@user_id/job_offer/received|jobs/job_title|@user_id/bookmark|'
    'api/search/specialty\?name=re|api/companies/id/bookmark|jobs\?job=|api/comapnies/id/follow_button|api/search/product'
)
condition_step3 = (
    ~condition_step1 & ~condition_step2 &
    (log['URL'].str.contains(pattern_step3) | log['URL'].isin(['jobs', 'people', 'companies', 'job']))
)

step3 = log[condition_step3 & log['user_uuid'].isin(step2_users)]
step3_users = set(step3['user_uuid'])

# Stage 4
pattern_step4 = (
    'jobs/id/id_title|people\?company|people\?rel=1|api/ask-manager/id|api/jobs/id/other_jobs\?offset=0&limit=5|api/jobs/id/other_jobs|'
    'jobs/id/bookmark|api/jobs/id/follow_button'
)
condition_step4 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & log['URL'].str.contains(pattern_step4)
)
step4 = log[condition_step4 & log['user_uuid'].isin(step3_users)]
step4_users = set(step4['user_uuid'])

# Stage 5
condition_except_step5 = log['URL'].isin(['api/users/id/experience/form?type=apply'])
pattern_step5 = (
    'continue\?next=/@|continue\?next=/jobs/|api/references|jobs/id/apply/step|api/jobs/id/template_oneclick'
)
condition_step5 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & ~condition_step4 & (log['URL'].str.contains(pattern_step5) | condition_except_step5)
)
step5 = log[condition_step5 & log['user_uuid'].isin(step4_users)]
step5_users = set(step5['user_uuid'])

# Stage 6
pattern_step6 = '@user_id/applications|apply_progress|api/remove_application/id|jobs/id/apply/complete'
condition_step6 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & ~condition_step4 & ~condition_step5 & log['URL'].str.contains(pattern_step6)
)
step6 = log[condition_step6 & log['user_uuid'].isin(step5_users)]
step6_users = set(step6['user_uuid'])

In [23]:
step1['step'] = 'step1'
step2['step'] = 'step2'
step3['step'] = 'step3'
step4['step'] = 'step4'
step5['step'] = 'step5'
step6['step'] = 'step6'

/var/folders/lv/_sw1hk9x557bwq1stf_h_8rw0000gn/T/ipykernel_49316/2607383285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step1['step'] = 'step1'
/var/folders/lv/_sw1hk9x557bwq1stf_h_8rw0000gn/T/ipykernel_49316/2607383285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step2['step'] = 'step2'
/var/folders/lv/_sw1hk9x557bwq1stf_h_8rw0000gn/T/ipykernel_49316/2607383285.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [24]:
funnel_logs = pd.concat([step1, step2, step3, step4, step5, step6], ignore_index=True)

## 전처리

In [25]:
# timestamp 데이터타입 변경
funnel_logs['timestamp'] = pd.to_datetime(funnel_logs['timestamp'].str.replace(" UTC", "", regex=False), errors='coerce')

In [26]:
funnel_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3082005 entries, 0 to 3082004
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_uuid      object        
 1   URL            object        
 2   timestamp      datetime64[ns]
 3   date           object        
 4   response_code  int64         
 5   method         object        
 6   step           object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 164.6+ MB


## Outlier Removal
- Remove cases where the total funnel usage duration is 0 minutes or exceeds 6 months

In [ ]:
# user_id별로 timestamp 정렬
funnel_logs = funnel_logs.sort_values(['user_uuid', 'timestamp'])

In [ ]:
# 유저별로 단계별 체류시간
# .shift(-1): 다음 값을 현재 줄에 넣는 pandas 문법
funnel_logs['next_timestamp'] = funnel_logs.groupby('user_uuid')['timestamp'].shift(-1)

# NaN (이탈 유저의 마지막 단계)은 현재 timestamp로 채워서 time_diff = 0으로 만들기
funnel_logs['next_timestamp'] = funnel_logs['next_timestamp'].fillna(funnel_logs['timestamp'])

# 분 단위로 계산 : dt.total_senconds()에 1분은 60초
funnel_logs['time_diff'] = (funnel_logs['next_timestamp'] - funnel_logs['timestamp']).dt.total_seconds() / 60

# 인덱스 정리
funnel_logs.reset_index(inplace=True)

In [ ]:
# 유저별 전체 이용기간
# 유저별 단계별 체류시간(time_diff)의 합
user_whole_time = funnel_logs.groupby('user_uuid')['time_diff'].sum().reset_index()
user_whole_time

In [ ]:
# 이상치 구하기
outliers = user_whole_time[
    (user_whole_time['time_diff'] == 0) |
    (user_whole_time['time_diff'] >= 259200)
    ]

# 이상치 개수 출력
print(f"이상치의 개수: {outliers.shape[0]}개")

In [ ]:
# 이상치에 해당하는 user_uuid를 제외한 데이터만 필터링
filtered_user_whole_time = user_whole_time[~user_whole_time['user_uuid'].isin(outliers['user_uuid'])]

# 결과 확인
print(f"이상치가 제외된 데이터의 개수: {filtered_user_whole_time.shape[0]}개")

In [ ]:
# filtered_user_whole_time에 포함된 user_uuid 목록
valid_user_uuids = filtered_user_whole_time['user_uuid']

# funnel_logs에서 valid_user_uuids에 해당하는 유저만 필터링
filtered_funnel_logs = funnel_logs[funnel_logs['user_uuid'].isin(valid_user_uuids)]

# 결과 확인
print(f"필터링된 funnel_logs의 데이터 개수: {filtered_funnel_logs.shape[0]}")

In [ ]:
print(f"기존 funnel_logs에서 {funnel_logs.shape[0]-filtered_funnel_logs.shape[0]:,}개 user_uuid 제거됨")

In [ ]:
filtered_funnel_logs.shape

In [ ]:
# 데이터 추출
save_path = '/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/중급 분석 프로젝트/'
data_name = 'filtered_funnel_logs_logs.csv'
data_path = save_path + data_name
filtered_funnel_logs.to_csv(data_path, index=False)

# 4. 1단계 세그먼트 분류 및 최종 퍼널 데이터 생성

In [ ]:
data_path = '/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/중급 분석 프로젝트/filtered_funnel_logs_logs.csv'
log = pd.read_csv(data_path, index_col=0)

## 1단계 세그먼트 분류

In [ ]:
# signup 유저, joboffer 유저, 마케팅 경로 접속 유저, 단순 접속 유저로 구분

pattern_step1_signup = 'signup'
pattern_step1_joboffer = 'api/job_offer'
pattern_step1_marketing = 'setting\?utm_source|user_received'
pattern_step1_etc = 'api/recommend_specialty|widget|api/current_guided_action/id'

condition_signup = log['URL'].str.contains(pattern_step1_signup)
condition_joboffer = log['URL'].str.contains(pattern_step1_joboffer)
condition_marketing = log['URL'].str.contains(pattern_step1_marketing)
condition_etc = log['URL'].str.contains(pattern_step1_etc)

log['step1_segment'] = np.select([condition_signup, condition_joboffer, condition_marketing, condition_etc],
                                 ['signup', 'joboffer', 'marketing', 'etc'], default='none')

In [ ]:
log['step1_segment'].value_counts()

In [ ]:
# signup 유저만 필터링
log_from_signup = log[log['step1_segment'].isin(['signup', 'none'])]
log_from_signup

## 1단계 signup 유저 대상, 순서대로 이행한 유저 퍼널 도출

In [ ]:
log_steps = log_from_signup.dropna(subset=['step']).sort_values(['user_uuid', 'timestamp'])
user_step_seq = log_steps.groupby('user_uuid')['step'].apply(list).reset_index()

valid_funnel = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']

# 순서대로 잘 가다가 중간에 멈춘 유저 → 포함 예) ['step1', 'step2'] → OK
# 순서 꼬인 유저 → 제외 예) ['step1', 'step3', 'step2'] → X
# step1조차 없는 유저 → 제외

def is_ordered_prefix(seq, valid_funnel):
    index = 0
    for step in seq:
        if index < len(valid_funnel) and step == valid_funnel[index]:
            index += 1
        elif step in valid_funnel[index:]:
            # 올바르지 않은 순서로 다음 step 등장 → 탈락
            return False
    return index > 0  # 최소 step1은 있어야 유효

user_step_seq['valid'] = user_step_seq['step'].apply(lambda x: is_ordered_prefix(x, valid_funnel))

# 조건에 맞는 유저만 필터
strict_users = set(user_step_seq[user_step_seq['valid']]['user_uuid'])
strict_log = log_from_signup[log_from_signup['user_uuid'].isin(strict_users)]

In [ ]:
strict_log.shape

In [ ]:
strict_log.head(2)

In [ ]:
#1단계
pattern_step1 = 'setting\?utm_source|api/job_offer|user_received|signup|api/recommend_specialty|widget|api/current_guided_action/id'
condition_step1 = strict_log['URL'].str.contains(pattern_step1)
step1 = strict_log[condition_step1]
step1_users = set(step1['user_uuid'])

#2단계
condition_except_step5 = strict_log['URL'].isin(['api/users/id/experience/form?type=apply'])
pattern_step2 = (
    'email_verify\?code|api/users/id/specialty|api/users/id/form|@user_id/resume/step|api/users/id/career/id|api/users/id/project|'
    'api/users/id/resume/step|@user_id/resume|@user_id|api/users/id|api/guided_action/add|api/projects/id/media/add|api/users/id/template|'
    'api/media/id/form|api/project/form_data/media|api/verify/education/id|verify_phone'
)
condition_step2 = (
    ~condition_step1 & ~condition_except_step5 &
    (strict_log['URL'].str.contains(pattern_step2) | strict_log['URL'].isin(['setting']))
)
step2 = strict_log[condition_step2 & strict_log['user_uuid'].isin(step1_users)]
step2_users = set(step2['user_uuid'])

#3단계
pattern_step3 = (
    'timeline|suggest\?q|api/jobs/job_title|jobs\?specialty|api/jobs/collections/template|'
    'jobs\?location|api/specialties/id/follow_button|user_filter|search/companies|jobs\?page=&job|help/id|api/post/id|'
    'people\?school|people\?job=1&location|api/companies/id/view|companies/company_id/jobs|companies/company_id|widget|'
    'api/companies/id/member_list|@user_id/job_offer/received|jobs/job_title|@user_id/bookmark|'
    'api/search/specialty\?name=re|api/companies/id/bookmark|jobs\?job=|api/comapnies/id/follow_button|api/search/product'
)
condition_step3 = (
    ~condition_step1 & ~condition_step2 &
    (strict_log['URL'].str.contains(pattern_step3) | strict_log['URL'].isin(['jobs', 'people', 'companies', 'job']))
)

step3 = strict_log[condition_step3 & strict_log['user_uuid'].isin(step2_users)]
step3_users = set(step3['user_uuid'])

#4단계
pattern_step4 = (
    'jobs/id/id_title|people\?company|people\?rel=1|api/ask-manager/id|api/jobs/id/other_jobs\?offset=0&limit=5|api/jobs/id/other_jobs|'
    'jobs/id/bookmark|api/jobs/id/follow_button'
)
condition_step4 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & strict_log['URL'].str.contains(pattern_step4)
)
step4 = strict_log[condition_step4 & strict_log['user_uuid'].isin(step3_users)]
step4_users = set(step4['user_uuid'])

#5단계
condition_except_step5 = strict_log['URL'].isin(['api/users/id/experience/form?type=apply'])
pattern_step5 = (
    'continue\?next=/@|continue\?next=/jobs/|api/references|jobs/id/apply/step|api/jobs/id/template_oneclick'
)
condition_step5 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & ~condition_step4 & (strict_log['URL'].str.contains(pattern_step5) | condition_except_step5)
)
step5 = strict_log[condition_step5 & strict_log['user_uuid'].isin(step4_users)]
step5_users = set(step5['user_uuid'])

#6단계
pattern_step6 = '@user_id/applications|apply_progress|api/remove_application/id|jobs/id/apply/complete'
condition_step6 = (
    ~condition_step1 & ~condition_step2 & ~condition_step3 & ~condition_step4 & ~condition_step5 & strict_log['URL'].str.contains(pattern_step6)
)
step6 = strict_log[condition_step6 & strict_log['user_uuid'].isin(step5_users)]
step6_users = set(step6['user_uuid'])

# 5. 퍼널 분석 기초

## 단계별 로그 수

In [ ]:
print(f'1단계(방문,진입) 로그 수 : {step1.shape[0]:,}')
print(f'2단계(이력서 작성) 로그 수 : {step2.shape[0]:,}')
print(f'3단계(탐색) 로그 수 : {step3.shape[0]:,}')
print(f'4단계(공고 조회) 로그 수 : {step4.shape[0]:,}')
print(f'5단계(지원서 작성) 로그 수 : {step5.shape[0]:,}')
print(f'6단계(지원 완료) 로그 수 : {step6.shape[0]:,}')

## 단계별 고유 유저 수

In [ ]:
print(f'1단계 유저 수 : {len(step1_users):,}')
print(f'2단계 유저 수 : {len(step2_users):,}')
print(f'3단계 유저 수 : {len(step3_users):,}')
print(f'4단계 유저 수 : {len(step4_users):,}')
print(f'5단계 유저 수 : {len(step5_users):,}')
print(f'6단계 유저 수 : {len(step6_users):,}')

## 단계별 전환율, 이탈률

In [ ]:
# 단계별 사용자 수 리스트로 정리
step_user_sets = [step1_users, step2_users, step3_users, step4_users, step5_users, step6_users]
step_user_counts = [len(users) for users in step_user_sets]

for i in range(1, len(step_user_counts)):
    prev = step_user_counts[i - 1]
    curr = step_user_counts[i]

    # 전환율
    conversion_rate = (curr / prev * 100) if prev else 0

    # 이탈률
    dropoff_rate = ((prev - curr) / prev * 100) if prev else 0

    print(f"{i} → {i+1}단계 전환율: {conversion_rate:.2f}%, 이탈률: {dropoff_rate:.2f}%")

In [ ]:
# 데이터 추출
save_path = '/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/중급 분석 프로젝트/'
data_name = 'log_signup_filtering.csv'
data_path = save_path + data_name
strict_log.to_csv(data_path, index=False)

# 6. 퍼널 단계별 현황 분석

In [ ]:
data_path = '/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/중급 분석 프로젝트/strict_log.csv'
log = pd.read_csv(data_path)

## 퍼널 2단계 관련

### 가설 1) 자신의 이력서에 대한 확인 및 수정이 잦은 유저의 경우, 이탈률이 낮을 것이다
- 2단계 로그가 많이 찍힌 유저일 수록 이탈률이 낮을 것이다

In [ ]:
# 2단계 로그 수를 기준으로 유저를 그룹화
step2_segment = step2.groupby('user_uuid')['step'].count().reset_index()
step2_segment.columns = ['user_uuid', 'step2_count']
step2_segment.describe()

In [ ]:
# Step2 로그 수 기준으로 그룹화
# qcut : 4분위수 기준으로 구분
step2_segment['segment'] = pd.qcut(step2_segment['step2_count'], q=3, labels=['low', 'mid', 'high'])

In [ ]:
# 고유유저가 도달한 최대 단계 계산
user_max_step = log.groupby('user_uuid')['step'].max().reset_index()
user_max_step.columns = ['user_uuid', 'max_step']
user_max_step['max_step'].value_counts()

In [ ]:
user_max_step['max_step'].value_counts()

In [ ]:
# 세그먼트와 이탈 여부 합치기
merged = pd.merge(step2_segment, user_max_step, on='user_uuid', how='inner')

# 세그먼트별로 max_step 분포 보기
merged['segment'].value_counts()

In [ ]:
# 세그먼트별로 최종 도달 단계 비율 비교
# pd.crosstab() : 두 개의 컬럼을 기준으로 빈도표(교차표)를 만들어주는 함수, pd.crosstab(row, column)
# normalize='index' : 각 행 기준으로 비율(%)로 바꾸는 것

pivot = pd.crosstab(merged['segment'], merged['max_step'], normalize='index')
pivot

In [ ]:
# 시각화
pivot.plot(kind='bar', stacked=True, figsize=(10, 5), colormap='viridis')

plt.title('세그먼트별 최종 도달 단계 비율')
plt.xlabel('Step2 로그 수 기준 세그먼트')
plt.ylabel('단계별 도달 비율')
plt.legend(title='최종 도달 단계', bbox_to_anchor=(1.05, 1), loc='upper left')  # 범례 정렬
plt.tight_layout()
plt.show()

```python
💡 세그먼트가 높아질수록 이탈률이 낮고, 최종 단계 도달률이 높다
💡 특히, low 그룹은 step3~step4 단계에서 대부분 이탈하고, high 그룹은 step6까지 가는 비율이 95%
💡 자신의 이력서에 대한 화인, 수정 빈도가 높을 유저일 수록 최종 단계에 도달하는 비율이 높다
```

### 가설 2) 개인 이력서에 대한 직접적인 입력/수정 로그가 많을수록 최종 단계에 도달할 확률이 높다
- 가설1과 달리 이력서에 대한 직접적인 기재, 수정 로그가 있어야함

In [ ]:
# 이력서/포트폴리오 관련 로그 수 확인
resume_logs = strict_log[strict_log["URL"].str.contains("resume|portfolio", na=False)]
print("이력서/포트폴리오 관련 전체 로그 수:", len(resume_logs))

In [ ]:
# step4(공고 클릭) & step6(지원 완료) 로그 추출
click_logs = strict_log[strict_log["step"] == "step4"].copy()
apply_logs = strict_log[strict_log["step"] == "step6"].copy()

# 유저별 최초 클릭 & 지원 시간 계산
click_time_df = click_logs.groupby("user_uuid")["timestamp"].min().reset_index()
click_time_df.rename(columns={"timestamp": "click_time"}, inplace=True)

apply_time_df = apply_logs.groupby("user_uuid")["timestamp"].min().reset_index()
apply_time_df.rename(columns={"timestamp": "apply_time"}, inplace=True)

# 병합 후 시간차 계산
click_to_apply_df = pd.merge(click_time_df, apply_time_df, on="user_uuid", how="inner")
click_to_apply_df["click_time"] = pd.to_datetime(click_to_apply_df["click_time"], errors="coerce")
click_to_apply_df["apply_time"] = pd.to_datetime(click_to_apply_df["apply_time"], errors="coerce")
click_to_apply_df["click_to_apply_time"] = (
    (click_to_apply_df["apply_time"] - click_to_apply_df["click_time"]).dt.total_seconds() / 60
)

In [ ]:
# ① 전체 체류 시간 계산
explore_time = (
    strict_log.groupby("user_uuid")["time_diff"]
    .sum().reset_index().rename(columns={"time_diff": "explore_time_total"})
)
# ② step6 도달 여부
step6_users = strict_log[strict_log["step"] == "step6"]["user_uuid"].unique()
explore_time["step6_도달"] = explore_time["user_uuid"].isin(step6_users).astype(int)

# ③ 최종 도달 퍼널 단계
final_step = (
    strict_log.sort_values(["user_uuid", "timestamp"])
    .groupby("user_uuid")["step"].last().reset_index().rename(columns={"step": "final_step"})
)

# ④ 이력서/포트폴리오 수정 횟수
resume_edit_count = (
    strict_log[strict_log["URL"].str.contains("resume|portfolio", na=False)]
    .groupby("user_uuid").size().reset_index(name="resume_edit_count")
)

# ⑤ 유저 기반 마스터 테이블 병합
user_step_df = (
    explore_time
    .merge(final_step, on="user_uuid", how="left")
    .merge(resume_edit_count, on="user_uuid", how="left")
    .merge(click_to_apply_df[["user_uuid", "click_to_apply_time"]], on="user_uuid", how="left")
)

# ⑥ 결측값 처리
user_step_df["resume_edit_count"] = user_step_df["resume_edit_count"].fillna(0).astype(int)

In [ ]:
# 1. 이력서 관련 로그 (resume 관련 URL 포함)
resume_keywords = ['resume', 'career', 'project', 'template', 'experience']
resume_logs = strict_log[strict_log['URL'].str.contains('|'.join(resume_keywords), na=False)]

resume_edit_count = resume_logs.groupby('user_uuid').size().reset_index(name='resume_edit_count')

# 2. 탐색 단계 총 체류시간 (step3에서만)
explore_logs = strict_log[strict_log['step'] == 'step3']
explore_time_total = explore_logs.groupby('user_uuid')['time_diff'].sum().reset_index(name='explore_time_total')

# 3. 클릭 후 지원까지 걸린 시간
click_to_apply_df.columns = ['user_uuid', 'click_to_apply_time', 'click_time', 'apply_time']

# 4. 유저별 최종 도달 퍼널 단계
final_step = strict_log.groupby('user_uuid')['step'].last().reset_index(name='final_step')

# 5. 병합하여 user_step_df 생성
from functools import reduce

dfs_to_merge = [resume_edit_count, explore_time_total, click_to_apply_df, final_step]
user_step_df = reduce(lambda left, right: pd.merge(left, right, on='user_uuid', how='outer'), dfs_to_merge)

# 확인
display(user_step_df.head())

# ✅ step6 도달 여부 부여
step6_users = strict_log[strict_log["step"] == "step6"]["user_uuid"].unique()
user_step_df["step6_도달"] = user_step_df["user_uuid"].isin(step6_users).astype(int)

In [ ]:
# 이력서 로그가 있는 user_uuid 수
resume_user_set = set(resume_logs["user_uuid"].unique())
print("이력서 관련 로그를 가진 고유 유저 수:", len(resume_user_set))

# user_step_df에 포함된 유저 중 얼마나 resume 로그를 갖고 있는가?
step_user_set = set(user_step_df["user_uuid"].unique())
print("user_step_df에 포함된 유저 수:", len(step_user_set))

# 교집합
print("이력서 로그 + 분석 대상 유저 교집합 수:", len(resume_user_set & step_user_set))

In [ ]:
from scipy.stats import chi2_contingency

# ① 이력서 수정 횟수 구간 분류
# bins = [0, 1, 3, 5, float("inf")]
bins = [0, 10, 20, 30, float("inf")]
labels = ["10회 미만", "10~19회", "20~29회", "30회 이상"]
user_step_df["resume_group"] = pd.cut(user_step_df["resume_edit_count"], bins=bins, labels=labels, right=False)

# ② groupby로 전환율 계산
group_summary = (
    user_step_df.groupby("resume_group")["step6_도달"]
    .agg(["count", "sum"])
    .rename(columns={"sum": "도달자 수"})
)
group_summary["전환율(%)"] = (group_summary["도달자 수"] / group_summary["count"] * 100).round(2)
print(group_summary)

# ③ 교차표 생성
contingency_table = pd.crosstab(user_step_df["resume_group"], user_step_df["step6_도달"])
print("\n📊 이력서 수정 횟수 그룹 vs Step6 도달 여부 교차표:")
print(contingency_table)

# ④ 카이제곱 검정
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"\n✅ 카이제곱 통계량: {chi2:.2f}")
print(f"🎓 자유도: {dof}")
print(f"📌 유의확률(p-value): {p:.5f}")

# ⑤ 시각화
plt.figure(figsize=(8, 6))
plt.bar(group_summary.index, group_summary["전환율(%)"])
plt.title("이력서 수정 횟수 그룹별 Step6 도달률")
plt.ylabel("전환율 (%)")
plt.ylim(0, 100)
plt.xlabel("이력서 수정 횟수 구간")
plt.show()

In [ ]:
# 🎯 이력서 수정 로그 보유 유저만 추출
resume_users = user_step_df[user_step_df["user_uuid"].isin(resume_user_set)].copy()

# 🧠 이력서 수정 횟수 기준 그룹화
def categorize_resume_count(x):
    if x == 0:
        return "10회"
    elif x <= 2:
        return "10~19회"
    elif x <= 4:
        return "20~29회"
    else:
        return "30회 이상"

resume_users["resume_group"] = resume_users["resume_edit_count"].fillna(0).apply(categorize_resume_count)

# 📊 그룹별 도달률 요약
resume_summary = (
    resume_users.groupby("resume_group")["step6_도달"]
    .agg(["count", "sum"])
    .rename(columns={"sum": "도달자 수"})
)
resume_summary["전환율(%)"] = (resume_summary["도달자 수"] / resume_summary["count"] * 100).round(2)

# 📈 카이제곱 검정
contingency_table = pd.crosstab(resume_users["resume_group"], resume_users["step6_도달"])
chi2, p, dof, expected = chi2_contingency(contingency_table)

In [ ]:
# 표 출력
display(resume_summary)

# 교차표 출력
print("📊 이력서 수정 횟수 그룹 vs Step6 도달 여부 교차표:")
print(contingency_table)

# 검정 결과
print(f"\n✅ 카이제곱 통계량: {chi2:.2f}")
print(f"🎓 자유도: {dof}")
print(f"📌 유의확률(p-value): {p:.5f}")

## 퍼널 3단계 관련

### 가설) 3단계 체류시간(탐색시간)이 짧을수록 다음단계로의 전환이 낮다

In [ ]:
# 이탈자id 데이터
droupoff_3 = step3_users - step4_users
len(droupoff_3)

# 이탈자만 있는 데이터
log_dropoff_3 = step3[step3['user_uuid'].isin(droupoff_3)]
# 이탈자 테이블 시간순으로 재정렬
log_dropoff_3 = log_dropoff_3.sort_values('timestamp',ascending=True)

In [ ]:
step3_dropoff_users = step3_users - step4_users
step3_survivor_users = step3_users & step4_users

step3_user_time = log[log['step']=='step3'].groupby('user_uuid')['time_diff'].mean().reset_index(name='avg_time_diff')

step3_user_time['group'] = step3_user_time['user_uuid'].apply(
    lambda x: 'dropoff' if x in step3_dropoff_users else(
        'survivor' if x in step3_survivor_users else 'other'
    )
)

In [ ]:
log_dropoff_3['timestamp'] = pd.to_datetime(log_dropoff_3['timestamp'].str.replace(" UTC", "", regex=False), errors='coerce')
log_dropoff_3['next_timestamp'] = pd.to_datetime(log_dropoff_3['next_timestamp'].str.replace(" UTC", "", regex=False), errors='coerce')

In [ ]:
# 페이지 체류시간
log_dropoff = log_dropoff_3.sort_values(['user_uuid','timestamp'])
log_dropoff['next_timestamp'] = log_dropoff.groupby('user_uuid')['timestamp'].shift(-1)
log_dropoff['stay_duration'] = (log_dropoff['next_timestamp'] - log_dropoff['timestamp']).dt.total_seconds()

# 극단적으로 긴값들 제거후 관측
q25 = step3_user_time['avg_time_diff'].quantile(0.25)
filtered_step3 = step3_user_time[step3_user_time['avg_time_diff'] <= q25]

In [ ]:
filtered_step3

In [ ]:
plt.figure(figsize=(13, 5))
sns.histplot(data=filtered_step3, x='avg_time_diff', hue='group', bins=30, kde=True, stat='density', common_norm=False)
plt.title('Step3 – Avg Page Time per User (Outliers Removed)')
plt.xlabel('Avg Time Diff')
plt.ylabel('Density')
plt.show()

In [ ]:
# 이상치를 제거하지않은 데이터(전체 step3 데이터)에서 전환자와 이탈자간의 체류시간차이
# 비모수 검정
# 가설: 이탈자의 체류시간이 전환자의 체류시간보다 유의미하게 짧다

from scipy.stats import mannwhitneyu

# dropoff / survivor 그룹 데이터 추출
dropoff = step3_user_time[step3_user_time['group'] == 'dropoff']['avg_time_diff']
survivor = step3_user_time[step3_user_time['group'] == 'survivor']['avg_time_diff']

# Mann-Whitney U 검정 실행
stat, p_value = mannwhitneyu(dropoff, survivor, alternative='less')

# 결과 출력
print(f"Mann-Whitney U 통계량: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("→ 이탈자의 체류시간이 전환자의 체류시간보다 통계적으로 유의미하게 짧습니다.")
else:
    print("→ 이탈자의 체류시간이 전환자의 체류시간보다 통계적으로 유의미하게 짧지 않습니다")

## 퍼널 4단계 관련

### 가설1) 4단계 체류시간(탐색시간)이 짧을수록 다음단계로의 전환이 낮다

In [ ]:
droupoff_4 = step4_users - step5_users
len(droupoff_4)
# 이탈자만 있는 데이터
log_dropoff_4 = step4[step4['user_uuid'].isin(droupoff_4)]
# 이탈자 테이블 시간순으로 재정렬
log_dropoff_4 = log_dropoff_4.sort_values('timestamp',ascending=True)

In [ ]:
log_dropoff_4['timestamp'] = pd.to_datetime(log_dropoff_4['timestamp'].str.replace(" UTC", "", regex=False), errors='coerce')
log_dropoff_4['next_timestamp'] = pd.to_datetime(log_dropoff_4['next_timestamp'].str.replace(" UTC", "", regex=False), errors='coerce')

In [ ]:
# 페이지 체류시간
log_dropoff_4 = log_dropoff_4.sort_values(['user_uuid','timestamp'])
log_dropoff_4['next_timestamp'] = log_dropoff_4.groupby('user_uuid')['timestamp'].shift(-1)
log_dropoff_4['stay_duration'] = (log_dropoff_4['next_timestamp'] - log_dropoff_4['timestamp']).dt.total_seconds()

In [ ]:
# 전환여부에 따라 dropoff/survivor 그룹 나누기
step4_dropoff_users = step4_users - step5_users
step4_survivor_users = step4_users & step5_users

step4_user_time = log[log['step']=='step4'].groupby('user_uuid')['time_diff'].mean().reset_index(name='avg_time_diff')

step4_user_time['group'] = step4_user_time['user_uuid'].apply(
    lambda x: 'dropoff' if x in step4_dropoff_users else(
        'survivor' if x in step4_survivor_users else 'other'
    )
)
# dropoff/survivor만 남기기기
step4_user_time = step4_user_time[step4_user_time['group'].isin(['dropoff','survivor'])]

# 극단적인 값 제거 후 보기
q25 = step4_user_time['avg_time_diff'].quantile(0.25)
filtered_step4 = step4_user_time[step4_user_time['avg_time_diff'] <= q25]

print(filtered_step4.groupby('group')['avg_time_diff'].describe())

In [ ]:
plt.figure(figsize=(13, 5))
sns.histplot(data=filtered_step4, x='avg_time_diff', hue='group',
             bins=30, kde=True, stat='density', common_norm=False)
plt.title('Step4 – Avg Page Time per User (Outliers Removed)')
plt.xlabel('Avg Time Diff')
plt.ylabel('Density')
plt.show()

In [ ]:
# 이상치를 제거하지않은 데이터(전체 step4 데이터)에서 전환자와 이탈자간의 체류시간차이
# 비모수 검정

# 가설: 이탈자의 체류시간이 전환자의 체류시간보다 유의미하게 짧다

# dropoff / survivor 그룹 데이터 추출
dropoff = step4_user_time[step4_user_time['group'] == 'dropoff']['avg_time_diff']
survivor = step4_user_time[step4_user_time['group'] == 'survivor']['avg_time_diff']

# Mann-Whitney U 검정 실행
stat, p_value = mannwhitneyu(dropoff, survivor, alternative='less')

# 결과 출력
print(f"Mann-Whitney U 통계량: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("→ 이탈자의 체류시간이 전환자의 체류시간보다 통계적으로 유의미하게 짧습니다.")
else:
    print("→ 이탈자의 체류시간이 전환자의 체류시간보다 통계적으로 유의미하게 짧지 않습니다")

### 가설2) 4단계 공고 클릭수가 적을수록 다음단계로의 전환이 낮다

In [ ]:
# 이탈률이 가장 큰 4 -> 5 단계에서
# 공고클릭수와 다음단계전환율 비교
step4_job_click = log[(log['URL'].str.contains('jobs/id/id_title', na=False)) & (log['step'] == 'step4')]
click_counts = step4_job_click.groupby('user_uuid').size().reset_index(name='job_click_counts')

step4_users = set(log[log['step'] =='step4']['user_uuid'])
step5_users = set(log[log['step'] == 'step5']['user_uuid'])
dropoff_users = step4_users - step5_users

click_counts['is_dropoff'] = click_counts['user_uuid'].apply(lambda x: 'dropoff' if x in dropoff_users else 'survivor')

# 평균 비교
print(click_counts.groupby('is_dropoff')['job_click_counts'].describe())

In [ ]:
# 클릭률 그래프
plt.figure(figsize=(13, 5))
sns.histplot(data=click_counts, x='job_click_counts', hue='is_dropoff',
             bins=30, kde=True, stat='density', common_norm=False)
plt.title('Step4 – Click Counts')
plt.xlabel('Click counts')
plt.ylabel('Density')
plt.show()

In [ ]:
# 이상치를 제거하지않은 데이터(전체 step4 데이터)에서 전환자와 이탈자간의 클릭수차이
# 비모수 검정

# 가설: 이탈자의 클릭 수가 전환자의 클릭 수보다 유의미하게 적다

# dropoff / survivor 그룹 데이터 추출
dropoff = click_counts[click_counts['is_dropoff'] == 'dropoff']['job_click_counts']
survivor = click_counts[click_counts['is_dropoff'] == 'survivor']['job_click_counts']

# Mann-Whitney U 검정 실행
stat, p_value = mannwhitneyu(dropoff, survivor, alternative='less')

# 결과 출력
print(f"Mann-Whitney U 통계량: {stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("→ 이탈자의 클릭 수가 전환자의 클릭 수보다 유의미하게 적다")
else:
    print("→ 이탈자의 클릭 수가 전환자의 클릭 수보다 유의미하게 적지 않다")

### 가설3) 지원 공고에 대한 클릭이 잦은 유저의 경우, 이탈률이 낮을 것이다

In [ ]:
# 지원 공고에 대한 클릭 URL을 가진 로그만 필터링
jobclick_condition = step4['URL'].str.contains('jobs/id/id_title')
step4_jobclick = step4[(jobclick_condition)]

In [ ]:
# 로그 수 기준으로 그룹화
step4_jobclick_segment = step4_jobclick.groupby('user_uuid')['step'].count().reset_index()
step4_jobclick_segment.columns = ['user_uuid', 'step4_count']
step4_jobclick_segment

In [ ]:
# 지원공고 클릭 수를 기준으로 세그먼트 분류
step4_jobclick_segment['segment'] = pd.qcut(step4_jobclick_segment['step4_count'], q=3, labels=['low', 'mid', 'high'])
step4_jobclick_segment.head(2)

In [ ]:
# 기존 최고 단계 데이터와 합치기
merged = pd.merge(step4_jobclick_segment, user_max_step, on='user_uuid', how='inner')
merged['segment'].value_counts()

In [ ]:
# 세그먼트별 최종 도달 단계 비교
pivot = pd.crosstab(merged['segment'], merged['max_step'], normalize='index')
pivot

In [ ]:
# 시각화
pivot.plot(kind='bar', stacked=True, figsize=(10, 5), colormap='viridis')
plt.title('채용공고 세그먼트별 최종 도달 단계 비율')
plt.xlabel('step4 로그 수 기준 세그먼트')
plt.ylabel('단계별 도달 비율')
plt.legend(title='최종 도달 단계', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

```python
💡 세그먼트가 높아질수록 이탈률이 낮고, 최종 단계 도달률이 높다
💡 채용공고에 대한 클릭이 많은 유저일수록 최종 단계에 도달하는 비율이 높다
✅ 채용공고에 대한 클릭 수는 매우 강력한 예측 변수
🤔 당연해보이긴 하지만, 검증했다는 것에 의의!
```

### 가설4) 관련 공고 불러오기를 클릭한 유저의 경우, 이탈률이 낮을 것이다

In [ ]:
# 해당 URL 로그만 필터링
otherjobs_condition = step4['URL'].str.contains('jobs/id/other_jobs')
step4_otherjobs = step4[otherjobs_condition]
step4_otherjobs

In [ ]:
# 로그 수 기준으로 그룹화
step4_otherjobs_segment = step4_otherjobs.groupby('user_uuid')['step'].count().reset_index()
step4_otherjobs_segment.columns = ['user_uuid', 'step4_count']
step4_otherjobs_segment.describe()

In [ ]:
# step4 로그 수 기준으로 세그먼트
step4_otherjobs_segment['segment'] = pd.qcut(step4_otherjobs_segment['step4_count'], q=3, labels=['low', 'mid', 'high'])
step4_otherjobs_segment.head(2)

In [ ]:
# 최고단계 데이터와 합치기
merged = pd.merge(step4_otherjobs_segment, user_max_step, on='user_uuid', how='inner')
merged['segment'].value_counts()

In [ ]:
# 세그먼트별 최종 단계 도달 비율 비교
pivot = pd.crosstab(merged['segment'], merged['max_step'], normalize='index')
pivot

In [ ]:
# 시각화
pivot.plot(kind='bar', stacked=True, figsize=(10, 5), colormap='viridis')
plt.title('유사공고 클릭 세그먼트별 최종 도달 단계 비율')
plt.xlabel('Step4 로그 수 기준 세그먼트')
plt.ylabel('단계별 도달 비율')
plt.legend(title='최종 도달 단계', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 7. 유저 여정 기반 패턴 분석

## 유저별 URL 시퀀스 정리

In [ ]:
log['timestamp'] = pd.to_datetime(log['timestamp'].str.replace(" UTC", "", regex=False), errors='coerce')

In [ ]:
# 1) 유저별 URL 흐름 정리
user_flow = log.sort_values(by=['user_uuid', 'timestamp'])
# 2) 유저별 URL 시퀀스를 리스트로 만들기
user_url_step_path = user_flow.groupby('user_uuid').apply(lambda x: list(zip(x['step'], x['URL'], x['timestamp']))).reset_index(name='url_path')
user_url_step_path

## 6단계 도달 유저

In [ ]:
completed_users = user_url_step_path[user_url_step_path['url_path'].apply(lambda x:any(step == 'step6' for step, URL, timestamp in x))]
completed_users

### 랜덤 샘플링

In [ ]:
# random sampling을 통해 유저의 여정 확인
# .sample(1) : 무작위로 1행 뽑기
sample_user = completed_users.sample(1).iloc[0]
sample_user

In [ ]:
for step, url, timestamp in sample_user['url_path']:
     print(f"{timestamp} | {step}: {url}")

### 로그 수 기준 25%, 50%, 75% user

In [ ]:
# 각 유저의 로그 수 계산
completed_users['log_count'] = completed_users['url_path'].apply(len)

# 로그 수 기준 퍼센타일 유저 선택
# .value를 붙이면 Series 형식을 Numpy 배열로 변경
percentiles = [0.25, 0.5, 0.75]
quantile_values = completed_users['log_count'].quantile(percentiles).values

In [ ]:
# 각 퍼센타일에 해당하는 log_count 값과 가장 가까운 유저 선택
selected_users = []
for value in quantile_values:
    # 절대값 차이가 가장 적은 유저 1명 선택
    # .abs() : 절대값으로 거리를 계산
    # .argsort() : 절대값 기준으로 오름차순 정렬한 인덱스 순서를 반환
    # [:1] : 그중 가장 가까운 유저 1명만 선택
    closest_user = completed_users.iloc[(completed_users['log_count']-value).abs().argsort()[:1]]
    selected_users.append(closest_user)

In [ ]:
selected_users[1]

In [ ]:
# 결과정리 및 저장
selected_df = pd.concat(selected_users).drop_duplicates(subset='user_uuid')
selected_user_flows = {}

# .iterrows() : DF를 한 줄씩 돌면서 row로 가져오는 반복문
# _는 인덱스인데 사용하지 않아서 무시한 것
for _, row in selected_df.iterrows():
    user_id = row['user_uuid']
    url_path = row['url_path']
    selected_user_flows[user_id] = url_path
    print(f"\n=== {user_id} ({len(url_path)} logs) ===")
    for step, url, timestamp in url_path:
        print(f"{timestamp} | {step}: {url}")

```python
로그 수 25% 수준의 유저 행동 패턴
🧾 요약 (Summary)
- 총 로그 수 : 240개
- 이용 기간 : 8월 1일 ~ 9월 27일

🔍 퍼널 흐름 분석
- step3에서 company_id/job에 대한 로그가 많은 편
- 특정 회사, 직무, 사람에 대한 탐색이 잦은 것으로 보아 희망하는 공고가 나올 때까지 적극 탐색하는 것으로 보임

🗓️ 날짜별 로그
- 8월 1일, 약 23분 접속 : 최초 접속 및 회원가입 후, 직무 검색, 회사 검색, 특정 조건(지역, 직무 이름 등)을 넣는 검색 진행했고 채용 공고를 클릭한 후 유사공고까지 확인
- 8월 24일, 약 3분 접속 : 포트폴리오 작성, 직무 탐색, 공고 확인, 유사공고 확인 진행
- 8월 25일, 1분 접속 : 특정 회사의 직무, 채용 공고 확인
- 9월 5일, 약 16분 접속 : 포트폴리오 작성, 사람 검색, 직무 검색, 채용공고 확인, 유사공고 확인, 북마크 진행 >
- 9월 13일, 약 15분 접속 : 지역 조건 걸고 직무 탐색, 채용공고 확인, 지원서 작성 1단계 진입 후 다시 개인 포트폴리오로 복귀, 특정 상품 및 회사 검색, 포트폴리오 수정
											  지원서 단계별 작성, 지원 완료
- 9월 14일, 약 30분 접속 : 검색 및 채용공고 확인
- 9월 15일 : 짧은 접속을 통해 자신의 지원이력 확인
- 9월 27일 : 짧게 접속하여 지원서 작성단계 시작함 (사전에 저장해둔 지원서일지도)
```

```python
로그 수 50% 수준의 유저 행동 패턴
🧾 요약 (Summary)
- 총 로그 수 : 419개
- 이용 기간 : 6월 7일 ~ 6월 25일

🔍 퍼널 흐름 분석
- step1 회원가입 단계에서 개인 프로필 꼼꼼하게 작성하는 유저로 보임
- step4에서 채용공고를 확인한 후, 지원서 작성 및 완료까지 3분, 8분 등 짧게 소요
- 초기 접속에서는 포트폴리오 단계에서의 접속이 많았으나 후반부에는 곧바로 채용공고 확인 및 지원서 작성 및 완료로 이어짐

🗓️ 날짜별 로그
- 6월 7일, 약 14분 접속 : 회원가입 완료 후 개인 포트폴리오 보완 진행
- 6월 9일 & 6월 10일 약 1분 접속 : 접속 후 개인 포트폴리오 확인
- 6월 11일, 약 17분 접속 : 포트폴리오 보완
- 6월 12일, 약 1시간 접속 : 포트폴리오 보완이거나 특정 회사의 구성원 확인, 가입자에 대한 탐색을 많이 한 것처럼 보임
- 6월 16일, 약 30분 접속 : 포트폴리오 보완이거나 특정 인물들 탐색, 직무 검색, 채용공고 확인, 사람 검색, 채용공고 확인, 회사 검색, 채용공고 확인 진행, 지원서 작성 및 완료
												지원완료 후 다른 공고 확인 및 지원서 작성 및 완료, 지원내역 1개 삭제, 채용공고 추가 클릭
- 6월 25일 : 직무 및 회사 검색, 채용공고 확인, 유사공고 확인, 지원서 작성 (완료하지 않음), 유사공고 확인, 지원서 작성 및 완료, 추가 공고 확인, 지원서 작성 및 완료

```

```python
로그 수 75% 수준의 유저 행동 패턴
🧾 요약 (Summary)
- 총 로그 수 : 748개
- 이용 기간 : 6월 15일 ~ 9월 29일

🔍 퍼널 흐름 분석
- 하루에도 여러차례 접속하는 유저
- 장시간 접속 유저
- 채용공고 확인에 대한 로그 굉장히 많고, 지원서 작성을 장시간&장기간 진행

🗓️ 날짜별 로그
- 6월 15일, 10시간 접속 : 회원가입 후 포트폴리오 작성, 직무 검색 및 채용공고, 유사공고 확인, 지원서 작성 시작 > 다시 포트폴리오 작성으로 돌아온 후 직무 및
											상품 구체적 검색 반복 > 지원완료 > 추가검색 및 탐색, 지원서 작성 > 지원완료 를 반복
- 6월 16일, 1시간 30분 접속 후 1회 추가 접속 : 지원서 작성 > 채용공고 확인, 유사공고 확인 > 지원서 작성
- 6월 17일, 4시간 접속 : 곧 바로 자신의 지원서 세션으로 접속 > 지원서 작성 > 채용공고 확인 > 북마크 확인 > 지원서 작성
- 6월 19일, 5분 접속, 총 3회 접속 : 곧 바로 자신의 지원서 세션으로 접속 > 지원서 작성 > 채용공고 확인 > 유사공고 확인
- 6월 26일, 10분 접속, 총 2회 접속 : 회사 탐색, 지원서 작성 > 회사 북마크 > 회사 페이지 보기
- 6월 28일, 5분 접속 : 포트폴리오 확인 및 보완, 사람 검색
- 7월 2일, 여러차례 접속 : 채용공고 확인 > 포트폴리오 보완 > 지원서 작성 > 회사 및 직무, 공고 검색 및 확인 > 지원서 작성
- 7월 3일, 5분 접속 : 검색
- 7월 5일 : 지원서 작성...step3까지 간걸보면 완료일지도
- 7월 9일, 7월 27일, 8월 4일, 8월 7일, 9월 2일 : 잠시 접속하여 특정 회사 확인 및 포트폴리오 확인
- 9월 5일 : 공고 확인, 지원서 작성, 지원완료, 포트폴리오 보완
```

In [ ]:
!pip install plotly pandas
import plotly.express as px

# 유저별 로그를 모아줄 리스트
records = []

# selected_user_flows: {user_uuid: [(step, url, timestamp), ...]}
for user_id, url_path in selected_user_flows.items():
    for step, url, timestamp in url_path:
        records.append({
            'user_id': user_id,
            'step': step,
            'url': url,
            'timestamp': pd.to_datetime(timestamp)
        })

# 데이터프레임으로 변환
df_timeline = pd.DataFrame(records)

# step 정렬 순서 명시 (안 하면 알파벳 순)
step_order = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
df_timeline['step'] = pd.Categorical(df_timeline['step'], categories=step_order, ordered=True)

# 시각화
fig = px.line(df_timeline,
              x="timestamp",
              y="step",
              color="user_id",
              markers=True,
              title="유저별 퍼널 단계 타임라인",
              labels={"step": "퍼널 단계", "timestamp": "시간", "user_id": "유저"},
              hover_data=["url"])

fig.update_layout(yaxis=dict(categoryorder='array', categoryarray=step_order))

fig.show()

## 5단계 이탈 유저

In [ ]:
import re

def extract_max_step(path):
    steps = [int(re.search(r'\d+', step).group()) for step, _, _ in path]
    return max(steps)

dropoff_users = user_url_step_path[user_url_step_path['url_path'].apply(lambda path: extract_max_step(path) == 5)]

In [ ]:
dropoff_users.shape

In [ ]:
sample_dropoff_user = dropoff_users.copy()

# 각 유저의 로그 수 계산
sample_dropoff_user['log_count'] = sample_dropoff_user['url_path'].apply(len)

# 퍼센타일 계산
percentiles = [0.25, 0.5, 0.75]
quantile_values = sample_dropoff_user['log_count'].quantile(percentiles).values

In [ ]:
# 각 퍼센타일에 해당하는 log_count 값과 가장 가까운 유저 선택
selected_users = []
for value in quantile_values:
    # 절대값 차이가 가장 적은 유저 1명 선택
    # .abs() : 절대값으로 거리를 계산
    # .argsort() : 절대값 기준으로 오름차순 정렬한 인덱스 순서를 반환
    # [:1] : 그중 가장 가까운 유저 1명만 선택
    closest_user = sample_dropoff_user.iloc[(sample_dropoff_user['log_count']-value).abs().argsort()[:1]]
    selected_users.append(closest_user)

In [ ]:
# 결과정리 및 저장
selected_df = pd.concat(selected_users).drop_duplicates(subset='user_uuid')
selected_user_flows = {}

# .iterrows() : DF를 한 줄씩 돌면서 row로 가져오는 반복문
# _는 인덱스인데 사용하지 않아서 무시한 것
for _, row in selected_df.iterrows():
    user_id = row['user_uuid']
    url_path = row['url_path']
    selected_user_flows[user_id] = url_path
    print(f"\n=== {user_id} ({len(url_path)} logs) ===")
    for step, url, timestamp in url_path:
        print(f"{timestamp} | {step}: {url}")

In [ ]:
# 유저별 로그를 모아줄 리스트
records = []

# selected_user_flows: {user_uuid: [(step, url, timestamp), ...]}
for user_id, url_path in selected_user_flows.items():
    for step, url, timestamp in url_path:
        records.append({
            'user_id': user_id,
            'step': step,
            'url': url,
            'timestamp': pd.to_datetime(timestamp)
        })

# 데이터프레임으로 변환
df_timeline = pd.DataFrame(records)

# step 정렬 순서 명시 (안 하면 알파벳 순)
step_order = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
df_timeline['step'] = pd.Categorical(df_timeline['step'], categories=step_order, ordered=True)

# 시각화
fig = px.line(df_timeline,
              x="timestamp",
              y="step",
              color="user_id",
              markers=True,
              title="유저별 퍼널 단계 타임라인",
              labels={"step": "퍼널 단계", "timestamp": "시간", "user_id": "유저"},
              hover_data=["url"])

fig.update_layout(yaxis=dict(categoryorder='array', categoryarray=step_order))

fig.show()

## 4단계 이탈 유저

### 랜덤 샘플링

In [ ]:
def extract_max_step(path):
    steps = [int(re.search(r'\d+', step).group()) for step, _, _ in path]
    return max(steps)

dropoff_users = user_url_step_path[user_url_step_path['url_path'].apply(lambda path: extract_max_step(path) == 4)]

In [ ]:
dropoff_users.shape

In [ ]:
sample_dropoff_user = dropoff_users.sample(1).iloc[0]

In [ ]:
sample_dropoff_user

In [ ]:
for step, url, timestamp in sample_dropoff_user['url_path']:
    print(f"{timestamp} | {step}: {url}")

### 로그 수 기준 25%, 50%, 75% user

In [ ]:
sample_dropoff_user = dropoff_users.copy()

# 각 유저의 로그 수 계산
sample_dropoff_user['log_count'] = sample_dropoff_user['url_path'].apply(len)

# 퍼센타일 계산
percentiles = [0.25, 0.5, 0.75]
quantile_values = sample_dropoff_user['log_count'].quantile(percentiles).values

In [ ]:
# 각 퍼센타일에 해당하는 log_count 값과 가장 가까운 유저 선택
selected_users = []
for value in quantile_values:
    # 절대값 차이가 가장 적은 유저 1명 선택
    # .abs() : 절대값으로 거리를 계산
    # .argsort() : 절대값 기준으로 오름차순 정렬한 인덱스 순서를 반환
    # [:1] : 그중 가장 가까운 유저 1명만 선택
    closest_user = sample_dropoff_user.iloc[(sample_dropoff_user['log_count']-value).abs().argsort()[:1]]
    selected_users.append(closest_user)

In [ ]:
# 결과정리 및 저장
selected_df = pd.concat(selected_users).drop_duplicates(subset='user_uuid')
selected_user_flows = {}

# .iterrows() : DF를 한 줄씩 돌면서 row로 가져오는 반복문
# _는 인덱스인데 사용하지 않아서 무시한 것
for _, row in selected_df.iterrows():
    user_id = row['user_uuid']
    url_path = row['url_path']
    selected_user_flows[user_id] = url_path
    print(f"\n=== {user_id} ({len(url_path)} logs) ===")
    for step, url, timestamp in url_path:
        print(f"{timestamp} | {step}: {url}")

In [ ]:
# 유저별 로그를 모아줄 리스트
records = []

# selected_user_flows: {user_uuid: [(step, url, timestamp), ...]}
for user_id, url_path in selected_user_flows.items():
    for step, url, timestamp in url_path:
        records.append({
            'user_id': user_id,
            'step': step,
            'url': url,
            'timestamp': pd.to_datetime(timestamp)
        })

# 데이터프레임으로 변환
df_timeline = pd.DataFrame(records)

# step 정렬 순서 명시 (안 하면 알파벳 순)
step_order = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
df_timeline['step'] = pd.Categorical(df_timeline['step'], categories=step_order, ordered=True)

# 시각화
fig = px.line(df_timeline,
              x="timestamp",
              y="step",
              color="user_id",
              markers=True,
              title="유저별 퍼널 단계 타임라인",
              labels={"step": "퍼널 단계", "timestamp": "시간", "user_id": "유저"},
              hover_data=["url"])

fig.update_layout(yaxis=dict(categoryorder='array', categoryarray=step_order))

fig.show()

## 3단계 이탈 유저

### 랜덤 샘플링

In [ ]:
def extract_max_step(path):
    steps = [int(re.search(r'\d+', step).group()) for step, _, _ in path]
    return max(steps)

dropoff_users = user_url_step_path[user_url_step_path['url_path'].apply(lambda path: extract_max_step(path) == 3)]

In [ ]:
dropoff_users.shape

In [ ]:
sample_dropoff_user = dropoff_users.sample(1).iloc[0]
sample_dropoff_user

In [ ]:
for step, url, timestamp in sample_dropoff_user['url_path']:
    print(f"{timestamp} | {step}: {url}")

### 로그 수 기준 25%, 50%, 75% user

In [ ]:
sample_dropoff_user = dropoff_users.copy()

# 각 유저의 로그 수 계산
sample_dropoff_user['log_count'] = sample_dropoff_user['url_path'].apply(len)

# 퍼센타일 계산
percentiles = [0.25, 0.5, 0.75]
quantile_values = sample_dropoff_user['log_count'].quantile(percentiles).values

In [ ]:
# 각 퍼센타일에 해당하는 log_count 값과 가장 가까운 유저 선택
selected_users = []
for value in quantile_values:
    # 절대값 차이가 가장 적은 유저 1명 선택
    # .abs() : 절대값으로 거리를 계산
    # .argsort() : 절대값 기준으로 오름차순 정렬한 인덱스 순서를 반환
    # [:1] : 그중 가장 가까운 유저 1명만 선택
    closest_user = sample_dropoff_user.iloc[(sample_dropoff_user['log_count']-value).abs().argsort()[:1]]
    selected_users.append(closest_user)

In [ ]:
# 결과정리 및 저장
selected_df = pd.concat(selected_users).drop_duplicates(subset='user_uuid')
selected_user_flows = {}

# .iterrows() : DF를 한 줄씩 돌면서 row로 가져오는 반복문
# _는 인덱스인데 사용하지 않아서 무시한 것
for _, row in selected_df.iterrows():
    user_id = row['user_uuid']
    url_path = row['url_path']
    selected_user_flows[user_id] = url_path
    print(f"\n=== {user_id} ({len(url_path)} logs) ===")
    for step, url, timestamp in url_path:
        print(f"{timestamp} | {step}: {url}")

In [ ]:
# 유저별 로그를 모아줄 리스트
records = []

# selected_user_flows: {user_uuid: [(step, url, timestamp), ...]}
for user_id, url_path in selected_user_flows.items():
    for step, url, timestamp in url_path:
        records.append({
            'user_id': user_id,
            'step': step,
            'url': url,
            'timestamp': pd.to_datetime(timestamp)
        })

# 데이터프레임으로 변환
df_timeline = pd.DataFrame(records)

# step 정렬 순서 명시 (안 하면 알파벳 순)
step_order = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
df_timeline['step'] = pd.Categorical(df_timeline['step'], categories=step_order, ordered=True)

# 시각화
fig = px.line(df_timeline,
              x="timestamp",
              y="step",
              color="user_id",
              markers=True,
              title="유저별 퍼널 단계 타임라인",
              labels={"step": "퍼널 단계", "timestamp": "시간", "user_id": "유저"},
              hover_data=["url"])

fig.update_layout(yaxis=dict(categoryorder='array', categoryarray=step_order))

fig.show()